In [2]:
import sys
sys.path.append("..")

from src.train import train_model
from src.data import prep_data
from src.nets import Net

from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from torch import nn
import torch.nn.functional as F

from torchvision import models

In [3]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.batch = nn.BatchNorm2d(2)
#         self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(64 * 18 * 18, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 2)
#         self.drp = nn.Dropout(0.4)

#     def forward(self, x):
#         x = self.batch(x)
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.drp(x)
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.drp(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batch = nn.BatchNorm2d(2)
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2048, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 2)
        self.drp = nn.Dropout(0.3)

    def forward(self, x):
        x = self.batch(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.drp(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drp(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.drp(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.drp(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
net = Net()
# net = model_ft


epochs = 1 # for testing on laptop
criterion = CrossEntropyLoss()

# adding weight_decay is a form of L2 regularization.
# See: https://discuss.pytorch.org/t/simple-l2-regularization/139
optimizer = Adam(net.parameters(), weight_decay=1e-5)#, lr=1e-3)

train_model(epochs, net, criterion, optimizer, early_stop=False)

  0%|                                                                                                                                                  | 0/1 [00:00<?, ?it/s]

[ loss: 0.6730 | acc: 0.5421 | vloss: 0.6249 | vacc: 0.6608 ] 


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.21s/it]


In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# for param in model_ft.parameters():
#     param.requires_grad = False
    
model_ft.fc = nn.Linear(num_ftrs, 2)

In [ ]:
model = models.vgg16(pretrained=True)